In [9]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [2]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert

from script import request_tri
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [ ]:
# Charger le fichier CSV contenant les transactions
transactions = pd.read_csv("data/base.csv",sep=";",encoding="utf-8")

# Ajouter une colonne pour savoir si le bien est en zone inondable
transactions[['results','identifiant_tri','libelle_type_inondation','code_scenario']] = transactions.apply(
    lambda row: pd.Series(request_tri.check_inondable(row['latitude'], row['longitude'])),
    axis=1
)


In [39]:
# Sauvegarder le fichier avec les résultats

transactions['results'] = transactions['results'].replace({2: 0, 3: 0})

transactions.to_csv("transactions_with_zone_inondable.csv",sep=";",encoding="utf-8", index=False)

In [ ]:
# import swifter

# test=transactions.iloc[0:1000]
# test[['results', 'identifiant_tri', 'libelle_type_inondation', 'code_scenario']] = test.swifter.apply(
#     lambda row: pd.Series(request_tri.check_inondable(row['latitude'], row['longitude'])),
#     axis=1
#     )

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).


C:\Users\flori\AppData\Local\Temp\ipykernel_19932\2683548646.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[['results', 'identifiant_tri', 'libelle_type_inondation', 'code_scenario']] = test.swifter.apply(


In [27]:

# Étape 1 : Compter et afficher le nombre de transactions par commune
transaction_counts = transactions.groupby('nom_commune').size().sort_values(ascending=False)
print(transaction_counts)

nom_commune
Nice                        7718
Cannes                      2772
Toulon                      2684
Brest                       2026
Antibes                     2019
                            ... 
Montreuil                      1
Cadillac                       1
Monte                          1
Mont-Saxonnex                  1
Conflans-Sainte-Honorine       1
Length: 645, dtype: int64


In [41]:
filtered_transactions = transactions.groupby('nom_commune').filter(
    lambda group: len(group) >= 10 and group['results'].sum() > 0 and (group['results'] == 0).any()
)

In [34]:
# Étape 3 : Calculer la moyenne agrégée des prix par commune et par zone inondable
filtered_transactions['prix_m2'] = filtered_transactions['valeur_fonciere'] / filtered_transactions['surface_reelle_bati']

In [35]:
# Moyenne par commune et zone inondable
mean_prices = filtered_transactions.groupby(['nom_commune', 'results'])['prix_m2'].mean().unstack()

In [49]:
# Calcul du prix moyen total par commune
filtered_transactions['prix_m2'] = filtered_transactions['valeur_fonciere'] / filtered_transactions['surface_reelle_bati']

# Calcul du prix moyen total par commune
mean_total = transactions.groupby('nom_commune')['prix_m2'].mean()

# Calcul du prix moyen par commune et par zone inondable
mean_by_zone = filtered_transactions.groupby(['nom_commune', 'results'])['prix_m2'].mean().unstack()

# Ajouter les colonnes pour le prix moyen en zone non inondable (results = 0) et zone inondable (results = 1)
mean_by_zone = mean_by_zone.rename(columns={0: 'prix_moyen_non_inondable', 1: 'prix_moyen_inondable'})

# Ajouter la colonne pour les prix moyens totaux
mean_by_zone['prix_moyen_total'] = mean_total

# Calculer la différence entre les prix moyens
mean_by_zone['différence_inondable'] = mean_by_zone['prix_moyen_inondable'] - mean_by_zone['prix_moyen_non_inondable']

# Ajouter le nombre d'observations en zone inondable et non inondable
observations_count = filtered_transactions.groupby(['nom_commune', 'results']).size().unstack(fill_value=0)
observations_count = observations_count.rename(columns={0: 'nb_non_inondable', 1: 'nb_inondable'})

# Joindre ces informations au DataFrame 'mean_by_zone'
mean_by_zone = mean_by_zone.join(observations_count)

# Trier les communes par différence décroissante
result_table = mean_by_zone.sort_values(by='différence_inondable', ascending=False)

# Résultat final
print(result_table)
print(result_table.dtypes)

# Exporter le DataFrame final en CSV
result_table.to_csv('data/moyenne.csv', encoding="utf-8", sep=";", decimal=",", index=True)


results                 prix_moyen_non_inondable  prix_moyen_inondable  \
nom_commune                                                              
Carqueiranne                         4985.877172           9000.000000   
Nice                                 6797.640811          10713.955163   
Port-Vendres                         3053.529092           6752.030303   
Cagnes-sur-Mer                       5646.538284           8977.940631   
Dives-sur-Mer                        3590.560495           5410.173293   
...                                          ...                   ...   
Toulon                               4057.139106           2576.548860   
Sérignan                             4232.625174           2675.023332   
Roquebrune-sur-Argens                5915.697345           4040.059137   
Urrugne                              4996.460936           1776.940783   
Saint-Georges-d'Oléron               6492.978020           3210.256410   

results                 prix_moyen_to

In [5]:
import requests

url2 = "https://georisques.gouv.fr/api/v1/tri_zonage"  # Remplacez par l'URL de votre API réelle

params2 = {
    'latlon': f"{6.958845},{43.541189}",
}

response2 = requests.get(url2, params=params2)
response2.json()

{'results': 1,
 'data': [{'code_national_tri': '06DREAL20140006',
   'identifiant_tri': 'FRD_TRI_NICE',
   'libelle_tri': 'Nice Cannes Mandelieu',
   'cours_deau': 'riou_siagne',
   'typeInondation': {'code': '01', 'libelle': "débordement de cours d'eau"},
   'scenario': {'code': '04Fai', 'libelle': 'aléa de faible probabilité'},
   'date_arrete_pcb': '12/12/2012',
   'date_arrete_carte': '20/12/2013',
   'date_arrete_pcb_local': None,
   'date_arrete_prefet_parties_prenantes': None,
   'date_arrete_approbation': None,
   'date_arrete_national': None,
   'code_insee': '06029',
   'libelle_commune': 'CANNES'}],
 'response_code': 200,
 'message': None}